In [1]:
import math
import numpy as np
from scipy.integrate import solve_bvp


# CONSTANTS
M_w = 18e-3  
T_0 = 273.15  
R = 8.31446  
F = 96485.333  
P_ref = 101325  
T_ref = T_0 + 80  

# OPERATING CONDITIONS
U = np.arange(1.15, -0.05, -0.05)  
P_A = 1.5e5  
P_C = 1.5e5  
RH_A = 0.90  
RH_C = 0.90  
s_C = 0.12  
T_A = T_0 + 70  
T_C = T_0 + 70  
alpha_H2 = 1  
alpha_O2 = 0.21  


# ELECTROCHEMICAL PARAMETERS
def A(E, T):
    return math.exp(E / R * (1 / T_ref - 1 / T))  

def i_0_HOR(T):
    return 0.27e4 * A(16e3, T)  

def i_0_ORR(T, x_O2):
    P_C = 1  # Assuming P_C is a constant
    return 2.47e-4 * (x_O2 * P_C / P_ref) ** 0.54 * A(67e3, T)  

beta_HOR = 0.5  
beta_ORR = 0.5  
DeltaH = -285.83e3  
DeltaS_HOR = 0.104  
DeltaS_ORR = -163.3  

# MATERIAL PARAMETERS
L = [160, 10, 25, 10, 160]  
a_ACL = 1e7  
a_CCL = 3e7  
H_ec = 42e3  
H_ad = H_ec  
k_GDL = 1.6  
k_CL = 0.27  
k_PEM = 0.3  
s_im = s_C  
V_m = 1.02 / 1.97e3  
V_w = M_w / 0.978e3  
eps_i_CL = 0.3  
eps_p_GDL = 0.76  
eps_p_CL = 0.4 
kappa_GDL = 6.15e-12  
kappa_CL = 1e-13  
sigma_e_GDL = 1250  
sigma_e_CL = 350  
tau_GDL = 1.6  
tau_CL = 1.6 

# WATER CONSTITUTIVE RELATIONSHIPS
def P_sat(T):
    return np.exp(23.1963 - 3816.44 / (T - 46.13))  

def mu(T):
    return 1e-3 * np.exp(-3.63148 + 542.05 / (T - 144.15))  

# MODEL PARAMETERIZATION
def BV(i_0, a, T, beta, eta):
    return i_0 * a * (math.exp(beta * 2 * F / (R * T) * eta) - math.exp(-(1 - beta) * 2 * F / (R * T) * eta))  

def D(eps_p, tau, s, P, T):
    return eps_p / tau**2 * (1 - s)**3 * (T / T_ref)**1.5 * (P_ref / P)  

def D_H2(eps_p, tau, s, T):
    return 1.24e-4 * D(eps_p, tau, s, P_A, T)  

def D_O2(eps_p, tau, s, T):
    return 0.28e-4 * D(eps_p, tau, s, P_C, T)  

def D_H2O_A(eps_p, tau, s, T):
    return 1.24e-4 * D(eps_p, tau, s, P_A, T)  

def D_H2O_C(eps_p, tau, s, T):
    return 0.36e-4 * D(eps_p, tau, s, P_C, T)  

def f(lambda_):
    return lambda_ * V_w / (V_m + lambda_ * V_w) 

x_H2O_A = RH_A * P_sat(T_A) / P_A  
x_H2O_C = RH_C * P_sat(T_C) / P_C  
x_H2_A = alpha_H2 * (1 - x_H2O_A)  
x_O2_C = alpha_O2 * (1 - x_H2O_C)  




# AUXILIARY FUNCTIONS
def iff(cond, a, b):
    return np.where(cond, a, b)  

# MATERIAL CONSTITUTIVE RELATIONSHIPS
def k_ad(lambda_, lambda_eq, T):
    return iff(lambda_ < lambda_eq, 3.53e-5, 1.42e-4) * f(lambda_) * A(20e3, T)  

def s_red(s):
    return (s - s_im) / (1 - s_im)  

def gamma_ec(x_H2O, x_sat, s, T):
    return 2e6 * iff(x_H2O < x_sat, 5e-4 * s_red(s), 6e-3 * (1 - s_red(s))) * np.sqrt(R * T / (2 * np.pi * M_w)) 

def sigma_p(eps_i, lambda_, T):
    return eps_i ** 1.5 * 116 * np.maximum(0, f(lambda_) - 0.06) ** 1.5 * A(15e3, T)  

def sorption(RH):
    return 0.043 + 17.81 * RH - 39.85 * RH ** 2 + 36.0 * RH ** 3  

def D_lambda(eps_i, lambda_, T):
    return eps_i ** 1.5 * (3.842 * lambda_ ** 3 - 32.03 * lambda_ ** 2 + 67.74 * lambda_) / (lambda_ ** 3 - 2.115 * lambda_ ** 2 - 33.013 * lambda_ + 103.37) * 1e-10 * A(20e3, T)  # [m^2/s] diffusion coefficient of water in Nafion

def xi(lambda_):
    return 2.5 * lambda_ / 22  
def dpds(s):
    return 0.00011 * 44.02 * np.exp(-44.02 * (s - 0.496)) + 278.3 * 8.103 * np.exp(8.103 * (s - 0.496))  

def D_s(kappa, s, T):
    return kappa * (1e-6 + s_red(s) ** 3) / mu(T) * dpds(s)  

# INITIAL MESH
Lsum = np.cumsum(L)
Nd = len(L)  
x = np.interp(np.linspace(0, Nd, Nd * 10 + 1), np.arange(Nd + 1), Lsum)
x = np.sort(np.concatenate((x, Lsum[1:-1])))  

# SOLVER PREPARATION
sol = solve_bvp(odefun, bcfun, x, yinit)
options = {'max_nodes': int(1e3), 'tol': 1e-4, 'max_subintervals': 1e3}

# PARAMETER SWEEP
I = np.zeros(U.shape)
SOL = [None] * U.size
Np = U.size  # number of parameters in the sweep
Neq = sol.y.shape[0] // 2  # number of 2nd-order differential equations
for k in range(Np):
    sol = solve_bvp(odefun, bcfun, sol.x, sol.y, tol=1e-4, max_nodes=int(1e3))
    SOL[k] = sol
    I[k] = sol.y[1, -1] / 1e4  # current density in [A/cm^2]
IU = np.column_stack((I, U))

Nref = 2  # number of refinements for smoother curve plotting
domains = np.array([[1, 1, 0, 1, 1],
                    [0, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1],
                    [0, 1, 1, 1, 0],
                    [1, 1, 0, 1, 1],
                    [1, 1, 0, 0, 0],
                    [0, 0, 0, 1, 1],
                    [0, 0, 0, 1, 1]])
shift = 1e-10
for k in range(Np):
    x = []
    for m in range(Nd):
        xa = np.where(SOL[k].x == Lsum[m])[0][-1]
        xb = np.where(SOL[k].x == Lsum[m + 1])[0][0]
        N = xb - xa

        # grid refinement
        x = np.concatenate((x, np.interp(np.linspace(shift, 1 - shift, N * 2 ** Nref + 1), np.linspace(0, 1, N + 1),
                                          SOL[k].x[xa:xb])))

        # fill solution on inactive domains with NaN
        SOL[k].y[~np.kron(domains[:, m], np.ones(2)), xa:xb] = np.nan

    SOL[k].y, SOL[k].yp = solve_bvp(odefun, bcfun, x, SOL[k].y)
    SOL[k].x = x

#% SYSTEM OF 1ST-ORDER DIFFERENTIAL EQUATIONS
def odefun(x, y, subdomain):
    # READ POTENTIALS & FLUXES
    phi_e = y[0, :]
    j_e = y[1, :]
    phi_p = y[2, :]
    j_p = y[3, :]
    T = y[4, :]
    j_T = y[5, :]
    lambda_val = y[6, :]
    j_lambda = y[7, :]
    x_H2O = y[8, :]
    j_H2O = y[9, :]
    x_H2 = y[10, :]
    j_H2 = y[11, :]
    x_O2 = y[12, :]
    j_O2 = y[13, :]
    s = y[14, :]
    j_s = y[15, :]

    # ZERO-INITIALIZE ALL DERIVATIVES
    z = np.zeros_like(x)
    dphi_e = z
    dj_e = z
    dphi_p = z
    dj_p = z
    dT = z
    dj_T = z
    dlambda = z
    dj_lambda = z
    dx_H2O = z
    dj_H2O = z
    dx_H2 = z
    dj_H2 = z
    dx_O2 = z
    dj_O2 = z
    ds = z
    dj_s = z

    # IMPLEMENT THE SYSTEM OF DIFFERENTIAL EQUATIONS HERE

    return np.vstack((dphi_e, dj_e, dphi_p, dj_p, dT, dj_T, dlambda, dj_lambda, dx_H2O, dj_H2O,
                      dx_H2, dj_H2, dx_O2, dj_O2, ds, dj_s))


# Compute derivatives
if subdomain == 1:  # AGDL
    C = P_A / (R * T)
    dphi_e = -j_e / sigma_e_GDL
    dT = -j_T / k_GDL
    dx_H2O = -j_H2O / (C * D_H2O_A(eps_p_GDL, tau_GDL, s, T))
    dx_H2 = -j_H2 / (C * D_H2(eps_p_GDL, tau_GDL, s, T))
    dj_T = -j_e * dphi_e

elif subdomain == 2:  # ACL
    C = P_A / (R * T)
    x_sat = P_sat(T) / P_A
    lambda_eq = sorption(x_H2O / x_sat)
    S_ad = k_ad(lambda, lambda_eq, T) / (L(2) * V_m) * (lambda_eq - lambda)
    eta = phi_e - phi_p + T * DeltaS_HOR / (2 * F) + R * T / (2 * F) * log(x_H2 * P_A / P_ref)
    i = BV(i_0_HOR(T), a_ACL, T, beta_HOR, eta)
    S_F = i / (2 * F)
    dphi_e = -j_e / sigma_e_CL
    dphi_p = -j_p / sigma_p(eps_i_CL, lambda, T)
    dT = -j_T / k_CL
    dlambda = (-j_lambda + xi(lambda) / F * j_p) * V_m / D_lambda(eps_i_CL, lambda, T)
    dx_H2O = -j_H2O / (C * D_H2O_A(eps_p_CL, tau_CL, s, T))
    dx_H2 = -j_H2 / (C * D_H2(eps_p_CL, tau_CL, s, T))
    dj_e = -i
    dj_p = i
    dj_T = -j_e * dphi_e - j_p * dphi_p + i * eta - S_F * T * DeltaS_HOR + H_ad * S_ad
    dj_lambda = S_ad
    dj_H2O = -S_ad
    dj_H2 = -S_F

elif subdomain == 3:  # PEM
    dphi_p = -j_p / sigma_p(1, lambda, T)
    dT = -j_T / k_PEM
    dlambda = (-j_lambda + xi(lambda) / F * j_p) * V_m / D_lambda(1, lambda, T)
    dj_T = -j_p * dphi_p

elif subdomain == 4:  # CCL
    C = P_C / (R * T)
    x_sat = P_sat(T) / P_C
    S_ec = gamma_ec(x_H2O, x_sat, s, T) * C * (x_H2O - x_sat)
    lambda_eq = sorption(x_H2O / x_sat)
    S_ad = k_ad(lambda, lambda_eq, T) / (L(4) * V_m) * (lambda_eq - lambda)
    eta = -(DeltaH - T * DeltaS_ORR) / (2 * F) + R * T / (4 * F) * log(x_O2 * P_C / P_ref) - (phi_e - phi_p)
    i = BV(i_0_ORR(T, x_O2), a_CCL, T, beta_ORR, eta)
    S_F = i / (2 * F)
    dphi_e = -j_e / sigma_e_CL
    dphi_p = -j_p / sigma_p(eps_i_CL, lambda, T)
    dT = -j_T / k_CL
    dlambda = (-j_lambda + xi(lambda) / F * j_p) * V_m / D_lambda(eps_i_CL, lambda, T)
    dx_H2O = -j_H2O / (C * D_H2O_C(eps_p_CL, tau_CL, s, T))
    dx_O2 = -j_O2 / (C * D_O2(eps_p_CL, tau_CL, s, T))
    ds = -j_s * V_w / D_s(kappa_CL, s, T)
    dj_e = i
    dj_p = -i
    dj_T = -j_e * dphi_e - j_p * dphi_p + i * eta - S_F * T * DeltaS_ORR + H_ad * S_ad + H_ec * S_ec
    dj_lambda = S_F + S_ad
    dj_H2O = -S_ec - S_ad

elif subdomain == 5:  # CGDL
    C = P_C / (R * T)
    x_sat = P_sat(T) / P_C
    S_ec = gamma_ec(x_H2O, x_sat, s, T) * C * (x_H2O - x_sat)
    dphi_e = -j_e / sigma_e_GDL
    dT = -j_T / k_GDL
    dx_H2O = -j_H2O / (C * D_H2O_C(eps_p_GDL, tau_GDL, s, T))
    dx_O2 = -j_O2 / (C * D_O2(eps_p_GDL, tau_GDL, s, T))
    ds = -j_s * V_w / D_s(kappa_GDL, s, T)
    dj_T = -j_e * dphi_e + H_ec * S_ec
    dj_H2O = -S_ec
    dj_s = S_ec




SyntaxError: invalid syntax (3845627508.py, line 240)